deep learning model training

In [7]:
import pandas as pd

df_2023 = pd.read_csv('data/learningdata/testdata_2023.csv',encoding='utf-8')
df_2022 = pd.read_csv('data/learningdata/testdata_2022.csv',encoding='utf-8')
df_2021 = pd.read_csv('data/learningdata/testdata_2021.csv',encoding='utf-8')

In [3]:
df_2023.head()

,일시,호선,역번호,역명,주말,요일,통과호선수,미세먼지(pm10),시간,하차인원,공휴일여부,강수량(mm),최저기온,최고기온,주변대학수
0,2023-01-01,1,150,서울역,1,6,4,54.0,6,636,1,0.0,-4.3,3.8,0
1,2023-01-01,1,151,시청,1,6,2,54.0,6,247,1,0.0,-4.3,3.8,0
2,2023-01-01,1,152,종각,1,6,1,53.0,6,164,1,0.0,-4.3,3.8,0
3,2023-01-01,1,153,종로3가,1,6,3,53.0,6,118,1,0.0,-4.3,3.8,0
4,2023-01-01,1,154,종로5가,1,6,1,53.0,6,84,1,0.0,-4.3,3.8,0


In [4]:
# 데이터 전처리 함수
import numpy as np

def adjustDf(temp_df):
    # 일시를 월+일로 시간을 cos함수로 변환 -> day_of_yer_cos에 저장
	temp_df['일시'] = pd.to_datetime(temp_df['일시'])
	temp_df['day_of_year'] = temp_df['일시'].dt.dayofyear
	temp_df['day_of_year_cos'] = np.cos(2 * np.pi * temp_df['day_of_year'] / 365)
	temp_df = temp_df.drop(['일시','day_of_year','역명'],axis=1)
	temp_df['하차인원'] = temp_df['하차인원'].apply(np.log1p)
	return temp_df

In [8]:
df = adjustDf(df_2022)
df.head(5)

,호선,역번호,주말,요일,통과호선수,미세먼지(pm10),시간,하차인원,공휴일여부,강수량(mm),최저기온,최고기온,주변대학수,day_of_year_cos
0,1,150,1,5,4,29.0,6,6.329721,1,0.0,-10.2,2.3,0,0.999852
1,1,151,1,5,2,29.0,6,5.278115,1,0.0,-10.2,2.3,0,0.999852
2,1,152,1,5,1,28.0,6,4.919981,1,0.0,-10.2,2.3,0,0.999852
3,1,153,1,5,3,28.0,6,4.941642,1,0.0,-10.2,2.3,0,0.999852
4,1,154,1,5,1,28.0,6,4.394449,1,0.0,-10.2,2.3,0,0.999852
